In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from whisperx_detect import detect_lyrics
from langchain_prompts import extract_prompt
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from generate_imgs import generate_imgs
from generate_video import generate_video
from langchain_FAISS import extract_prompt_FAISS
import openai
import os

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


In [2]:
os.environ['OPENAI_API_KEY'] = ...
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
# DEFINIMOS LA CONEXIÓN CON CHATGPT
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [4]:
# DETECTAMOS LAS LETRAS CON WHISPERX
lyrics = detect_lyrics("audios/paint it black.mp3")

No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Detected language: en (0.78) in first 30s of audio...


In [8]:
lyrics

{'segments': [{'text': ' I see a red door and I want it painted black No colors anymore, I want them to turn black',
   'start': 5.257,
   'end': 25.709},
  {'text': " I see a line of cars and they're all painted black With flowers and my love hope never to come back I see people turn their heads and quickly look away Like a newborn baby it just happens every day",
   'start': 36.273,
   'end': 61.636},
  {'text': " I look inside myself and see my heart is black I see my red door, I must have it painted black Maybe then I'll fade away and I'll have to face the facts Not easy facing up when your whole world is black No more will my green seagull turn a deeper blue",
   'start': 63.02,
   'end': 91.91},
  {'text': ' I could not foresee this thing happening to you If I look hard enough into the setting sun My love will laugh with me before the morning comes I see a red door and I want it painted black No colors anymore, I want them to turn black',
   'start': 93.378,
   'end': 122.453},
 

In [7]:
# OBTENEMOS EL ARTISTA Y EL TÍTULO POSIBLES A PARTIR DE LAS LETRAS DETECTADAS
song_prompt = f"""Try to identify the song that the lyrics below are from. \
maybe all lyrics not match at all, but try to find the closest one. \
if you have the song, your answer should be exclusively the title of the song and the artist. \
the lyrics are delimited by triple backticks:
```{lyrics}```
"""

song_title = get_completion(song_prompt)
song_title

'"Paint It, Black" by The Rolling Stones.'

In [8]:
# OBTENEMOS LAS LETRAS COMPLETAS A PARTIR DEL TITULO Y ARTISTA OBTENIDOS
lyrics_prompt =f"""give me the full lyrics of {song_title}\
just provide the full lyrics, not anything else, like the title of the song or the artist,
and no extra comments.
"""

response_lyrics = get_completion(lyrics_prompt)
response_lyrics

"I see a red door and I want it painted black\nNo colors anymore I want them to turn black\nI see the girls walk by dressed in their summer clothes\nI have to turn my head until my darkness goes\n\nI see a line of cars and they're all painted black\nWith flowers and my love both never to come back\nI see people turn their heads and quickly look away\nLike a newborn baby it just happens every day\n\nI look inside myself and see my heart is black\nI see my red door and it has been painted black\nMaybe then I'll fade away and not have to face the facts\nIt's not easy facing up when your whole world is black\n\nNo more will my green sea go turn a deeper blue\nI could not foresee this thing happening to you\nIf I look hard enough into the setting sun\nMy love will laugh with me before the morning comes\n\nI see a red door and I want it painted black\nNo colors anymore I want them to turn black\nI see the girls walk by dressed in their summer clothes\nI have to turn my head until my darkness

In [9]:
final_lyrics = [x for x in response_lyrics.split("\n") if x]
final_lyrics

['I see a red door and I want it painted black',
 'No colors anymore I want them to turn black',
 'I see the girls walk by dressed in their summer clothes',
 'I have to turn my head until my darkness goes',
 "I see a line of cars and they're all painted black",
 'With flowers and my love both never to come back',
 'I see people turn their heads and quickly look away',
 'Like a newborn baby it just happens every day',
 'I look inside myself and see my heart is black',
 'I see my red door and it has been painted black',
 "Maybe then I'll fade away and not have to face the facts",
 "It's not easy facing up when your whole world is black",
 'No more will my green sea go turn a deeper blue',
 'I could not foresee this thing happening to you',
 'If I look hard enough into the setting sun',
 'My love will laugh with me before the morning comes',
 'I see a red door and I want it painted black',
 'No colors anymore I want them to turn black',
 'I see the girls walk by dressed in their summer cl

In [10]:
# OBTENEMOS EL PROMPT EN BASE AL ESTILO ELEGIDO
image_prompt = extract_prompt(style='cyberpunk with dark tones')
image_prompt

'A futuristic cityscape with neon lights and heavy rain, in a cyberpunk style with dark tones.'

In [16]:
# OBTENEMOS EL PROMPT EN BASE AL ESTILO ELEGIDO UTILIZANDO EMBEDDINGS CON FAISS
image_prompt_faiss = extract_prompt_FAISS(style='cyberpunk with dark tones')
image_prompt_faiss = image_prompt_faiss[8:]
image_prompt_faiss

'a highly detailed and high technology alien landscape, dark cold tones colors theme, dramatic lighting, centered, corals, plume made of geometry, water texture, extremely detailed digital painting, sharp focus in the style of android jones, artwork of a futuristic artificial intelligence superstar with frames made of detailed circuits, mystical colors, rim light, octane render, 8 k, stunning scene, raytracing, under water visual distortion, dark tones colors, trending on artstation'

In [17]:
# DEFINIMOS UN OBJETO LANGCHAIN CAPAZ DE ALMACENAR LA MEMORIA DE LA CONVERSACIÓN
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", 
                 temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

# CREAMOS TODOS LOS PROMPTS PARA CADA FRASE DE LA CANCIÓN 
prompts_list = []
cont = 0

for lyrics in final_lyrics:    
    generation_prompt = f"""
    You are a prompt engineer for stable diffusion images generation. You are tasked with creating a prompt adapted to the following lyrics of a song:
    {lyrics}

    You have to create the prompt using this template: {image_prompt_faiss}

    don't modify the properties of the template, just modify the main object that is being
    created in the template, and use instead the lyrics provided to create a useful prompt that matches 
    the style of the template.
    the prompt that you provide must be brief and concise, describing the scene that the lyrics are about, not modifying at all the template.

    for example, if the template is creating a panda riding a bike, and the lyrics provided are about a dog,
    you must delete the panda riding a bike, and use instead the dog to create a useful prompt that matches the style of the template.

    in case the lyrics provided are not enough to create a prompt because they are expressions like "la la la" or "oh oh oh" or "yeah yeah yeah" and so on,
    try to create a prompt based on the previous lyrics provided on the context of the conversation that are not expressions like that.

    it's so important that you don't provide any additional comments or anything else, just the prompt.
    don't say things like "this is the prompt" or "this is the prompt that i created" or "this is the prompt that i created based on the lyrics provided" or anything else.

    if you are not able to create a prompt for whatever reason, just force the creation of the prompt.
    """

    prompts_list.append(conversation.predict(input = generation_prompt))

    cont += 1

    if cont == 3:
        break

In [18]:
prompts_list

['A highly detailed and high technology alien landscape, with dark cold tones colors theme and dramatic lighting, centered around a red door that is being painted black. The scene features corals and a plume made of geometry, with water texture and extremely detailed digital painting in the style of android jones. The artwork of a futuristic artificial intelligence superstar with frames made of detailed circuits is present, with mystical colors and rim light. The scene is rendered in 8k with stunning raytracing and under water visual distortion, all while maintaining the dark tones colors and trending on artstation.',
 'A highly detailed and high technology alien landscape, with dark cold tones colors theme and dramatic lighting, centered around a world where all colors have turned black. The scene features corals and a plume made of geometry, with water texture and extremely detailed digital painting in the style of android jones. The artwork of a futuristic artificial intelligence su

In [19]:
# INICIALIZAMOS LA GENERACIÓN DE IMÁGENES
cont = 0
output_dir = str(input('Nombre de la carpeta: '))
if output_dir not in os.listdir('./'):
    os.mkdir(output_dir)

for prompt in prompts_list:
    generate_imgs(prompt, num_imgs=1, cont=cont, output_dir=output_dir)
    cont += 1

In [20]:
#CREAMOS EL VIDEO A PARTIR DE LAS IMÁGENES GENERADAS
archivos = os.listdir(output_dir)
imagenes = [os.path.join(output_dir, imagen) for imagen in archivos]

nombre_video = str(input('Nombre del video: '))
salida = f'./{output_dir}/{nombre_video}.mp4'

generate_video(imagenes, salida, fps=60, espera=2)